# Binary Availability Table

In [1]:
import os, json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf
from tools import CreateLinkAdmin
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

### Merge crop data

In [4]:
# Merge crop data
countries_ISO = ['SO','MW','KE','BF','ML','TD','ZA','SN']
container = []
for name in countries_ISO:
    df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % name)
    # Select the calibrated data
    df = df[df['gscd_code'] == 'calibrated']
    container.append(df)
df = pd.concat(container, axis=0).reset_index(drop=True)

In [5]:
df

,fnid,country,country_code,admin_1,admin_2,name,product,season_name,growing_year,growing_month,harvest_year,harvest_month,crop_production_system,indicator,value,gscd_code
0,SO1990A21201,Somalia,SO,Woqooyi Galbeed,Hargeysa,Hargeysa,Maize,Gu,2012,03-01,2012,08-01,agro_pastoral,area,420.000000,calibrated
1,SO1990A21201,Somalia,SO,Woqooyi Galbeed,Hargeysa,Hargeysa,Maize,Gu,2012,03-01,2012,08-01,agro_pastoral,production,210.000000,calibrated
2,SO1990A21201,Somalia,SO,Woqooyi Galbeed,Hargeysa,Hargeysa,Maize,Gu,2012,03-01,2012,08-01,agro_pastoral,yield,0.500000,calibrated
3,SO1990A21201,Somalia,SO,Woqooyi Galbeed,Hargeysa,Hargeysa,Maize,Gu,2013,03-01,2013,08-01,agro_pastoral,area,2500.000000,calibrated
4,SO1990A21201,Somalia,SO,Woqooyi Galbeed,Hargeysa,Hargeysa,Maize,Gu,2013,03-01,2013,08-01,agro_pastoral,production,1750.000000,calibrated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81674,SN2008A21703,Senegal,SN,Saint-Louis,Saint-Louis,Saint-Louis,Millet,Main season,2015,06-01,2015,11-01,none,yield,0.250000,calibrated
81675,SN2008A21703,Senegal,SN,Saint-Louis,Saint-Louis,Saint-Louis,Sorghum,Main season,1982,06-01,1982,11-01,none,area,17.023698,calibrated
81676,SN2008A21703,Senegal,SN,Saint-Louis,Saint-Louis,Saint-Louis,Sorghum,Main season,1982,06-01,1982,11-01,none,production,0.680948,calibrated
81677,SN2008A21703,Senegal,SN,Saint-Louis,Saint-Louis,Saint-Louis,Sorghum,Main season,1982,06-01,1982,11-01,none,yield,0.040000,calibrated


In [9]:
df[['country','season_name','product','growing_month','harvest_month']].drop_duplicates().reset_index(drop=True)

,country,season_name,product,growing_month,harvest_month
0,Somalia,Gu,Maize,03-01,08-01
1,Somalia,Deyr,Maize,10-01,03-01
2,Somalia,Gu,Sorghum,03-01,08-01
3,Somalia,Deyr,Sorghum,10-01,03-01
4,Somalia,Gu-off,Maize,09-01,10-01
...,...,...,...,...,...
56,Senegal,Main season,Millet,06-01,11-01
57,Senegal,Main season,Rice,06-01,11-01
58,Senegal,Main season,Fonio,06-01,11-01
59,Senegal,Off season,Maize,02-01,07-01


In [54]:
df['country'].unique()

array(['Somalia', 'Malawi', 'Kenya', 'Burkina Faso', 'Mali', 'Chad',
       'South Africa', 'Senegal'], dtype=object)

In [52]:
sub = df[df['indicator'] == 'production']
table = sub.pivot_table(index=['country','season_name','product','growing_month','harvest_month','fnid'],columns='harvest_year',values='value',aggfunc='count')
table[table.isna()] = 0
table[table > 0] = 1
table = table.astype(np.int)
table.to_csv('./data/crop/binary_availability.csv')